<a href="https://colab.research.google.com/github/akkipant/Intell-Cyber-Security/blob/master/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

- If you are using Google Colab, you can simply install PySyft using the following command:
`! pip install syft`

- If you are using PySyft locally, the easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [1]:
!pip install syft

     |████████████████████████████████| 327kB 4.9MB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
     |████████████████████████████████| 81kB 26.8MB/s 
     |████████████████████████████████| 676.9MB 33kB/s 
     |████████████████████████████████| 399kB 51.1MB/s 
     |████████████████████████████████| 256kB 48.4MB/s 
     |████████████████████████████████| 2.7MB 48.0MB/s 
     |████████████████████████████████| 2.6MB 35.5MB/s 
     |████████████████████████████████| 460kB 39.4MB/s 
     |████████████████████████████████| 51kB 11.7MB/s 
     |████████████████████████████████| 266kB 45.6MB/s 
     |████████████████████████████████| 122kB 49.0MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076284 sha256=e3351d9e4df33fd8ea8ac6f776198e7d5f1bc2a766b6c535965d6fbba53adfff
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-li

In [2]:
import torch as th
import syft as sy

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'


In [0]:
hook = sy.TorchHook(th)  # this line of code creates an instance of th with its backend APIs modified with Syft functions

In [4]:
x = th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same
x

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [6]:
print(f'Type: {type(bob._objects)} \nValue: {bob._objects}')

Type: <class 'dict'> 
Value: {}


In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x =  x.send(bob) # send this data to bob

In [9]:
bob._objects

{98051831637: tensor([1, 2, 3, 4, 5])}

In [10]:
# What's the type of the pointer? and wht's the reason behind this type?
# What's its value?

print(f'Type: {type(x)} \nValue: {x}')

Type: <class 'torch.Tensor'> 
Value: (Wrapper)>[PointerTensor | me:62018491960 -> bob:98051831637]


In [11]:
x.location # where the Tensor located?

<VirtualWorker id:bob #objects:1>

In [12]:
x.id # the pointer ID at our machine

62018491960

In [13]:
x.id_at_location # the ID of the tensor at the remote worker

98051831637

In [14]:
x.owner

<VirtualWorker id:me #objects:0>

In [15]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [16]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id='alice')

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2,3,4])
x = x.send(bob, alice)

In [19]:
# notice what does the send fucntion on two workers return; a multi-pointer
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:98799554441 -> bob:62345816098]
	-> [PointerTensor | me:18847291052 -> alice:22103924603]

In [20]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
print(f'Type: {type(x.child)} \nValue: {x.child}')
print(f'Type: {type(x.child.child)} \nValue: {x.child.child}')

Type: <class 'syft.generic.pointers.multi_pointer.MultiPointerTensor'> 
Value: [MultiPointerTensor]
	-> [PointerTensor | me:98799554441 -> bob:62345816098]
	-> [PointerTensor | me:18847291052 -> alice:22103924603]
Type: <class 'dict'> 
Value: {'bob': [PointerTensor | me:98799554441 -> bob:62345816098], 'alice': [PointerTensor | me:18847291052 -> alice:22103924603]}


In [21]:
bob._objects

{62345816098: tensor([2, 3, 4])}

In [22]:
alice._objects

{22103924603: tensor([2, 3, 4])}

In [23]:
# try the .get() on the pointer
x = x.get()
x

[tensor([2, 3, 4]), tensor([2, 3, 4])]

In [24]:
alice._objects

{}

In [25]:
bob._objects

{}

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2, 5, 4])
x = x.send(bob, alice)

In [27]:
# try the .get(sum_results=True) on your pointer
sum = x.get(sum_results=True)
sum

tensor([ 4, 10,  8])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [29]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:49709233244 -> bob:56040267113]

In [30]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:91671948184 -> bob:51470714195]

In [31]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:22099018991 -> bob:14398567630]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [34]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [35]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

<VirtualWorker id:bob #objects:4>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1., 1],[0.5, 1],[1, 0],[0, 0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [38]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:32588783807 -> bob:33997209790]

In [39]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()
  weights.data.sub_(weights.grad * lr)
  weights.grad *= 0

  print(loss.get().data)

tensor(0.4827)
tensor(0.3459)
tensor(0.2529)
tensor(0.1893)
tensor(0.1455)
tensor(0.1150)
tensor(0.0935)
tensor(0.0781)
tensor(0.0668)
tensor(0.0584)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [41]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [43]:
bob._objects

{78620034758: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [45]:
bob._objects

{}

In [48]:
x.child.garbage_collect_data  # True by default

True

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [50]:
bob._objects

{5710626107: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [52]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [54]:
x

(Wrapper)>[PointerTensor | me:1217159386 -> bob:46337867863]

In [55]:
bob._objects

{46337867863: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [57]:
bob._objects  # some error from Jupyter is casuing this to appear!

{46337867863: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [59]:
bob._objects

{46337867863: tensor([1, 2, 3, 4, 5])}

In [60]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [62]:
bob._objects # notice that there is only a single tensor in bob

{57580543027: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [64]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [71]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).mean()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.2105)
tensor(0.1351)
tensor(0.0971)
tensor(0.0767)
tensor(0.0647)
tensor(0.0569)
tensor(0.0511)
tensor(0.0465)
tensor(0.0426)
tensor(0.0391)


# nothing was federated up to this point!

In [72]:
# let's reeat the previous experiment in a FL approach:

import torch as th
import syft as sy

hook = sy.TorchHook(th)  

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

bob = bob.clear_objects()
alice = alice.clear_objects()

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [74]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:69131605796 -> bob:42225915720],
  (Wrapper)>[PointerTensor | me:13879465539 -> bob:3269546125]),
 ((Wrapper)>[PointerTensor | me:47500937676 -> alice:59499618788],
  (Wrapper)>[PointerTensor | me:84265985465 -> alice:57948768463])]

In [0]:
def train(iterations=3):

    model = nn.Linear(2,1) # our model created locally
    opt = optim.SGD(params=model.parameters(), lr=0.1) # our optimizer created locally
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            # 1 zero the gradients
            opt.zero_grad()
            # 2 calculate predictions
            pred = model(_data)
            # 3 calculate loss -- MSE
            loss = ((pred - _target)**2).sum()
            # 4 calculate gradeints
            loss.backward()
            # 5 update weights
            opt.step()
            # return the model to the local machine
            model = model.get()
            
        print(loss.get())

In [76]:
train(3)

tensor(1.7139, requires_grad=True)
tensor(1.1051, requires_grad=True)
tensor(0.6980, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [77]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [80]:
bob._objects

{71945966497: tensor([1, 2, 3, 4, 5])}

In [81]:
alice._objects

{64007870237: (Wrapper)>[PointerTensor | alice:64007870237 -> bob:71945966497]}

In [0]:
y = x + x

In [83]:
y

(Wrapper)>[PointerTensor | me:39702391429 -> alice:67931012561]

In [84]:
bob._objects

{52774488289: tensor([ 2,  4,  6,  8, 10]),
 71945966497: tensor([1, 2, 3, 4, 5])}

In [85]:
alice._objects

{46517882926: (Wrapper)>[PointerTensor | alice:67931012561 -> bob:52774488289],
 64007870237: (Wrapper)>[PointerTensor | alice:64007870237 -> bob:71945966497],
 67931012561: (Wrapper)>[PointerTensor | alice:67931012561 -> bob:52774488289]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [87]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [89]:
z = x + y

TensorsNotCollocatedException: ignored

In [90]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:35054645057 -> bob:51558318506]

In [91]:
bob._objects

{32451111738: tensor([1, 2, 3, 4, 5]), 51558318506: tensor([1, 2, 3, 4, 5])}

In [92]:
alice._objects

{}

In [93]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [94]:
bob._objects

{32451111738: tensor([1, 2, 3, 4, 5])}

# Lesson: Pointer Chain Operations

In [95]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [97]:
bob._objects

{24043803146: tensor([1, 2, 3, 4, 5])}

In [98]:
alice._objects

{}

In [99]:
x.move(alice)

(Wrapper)>[PointerTensor | me:47823169695 -> alice:36837704319]

In [100]:
x

(Wrapper)>[PointerTensor | me:47823169695 -> alice:36837704319]

In [101]:
bob._objects

{}

In [102]:
alice._objects

{36837704319: tensor([1, 2, 3, 4, 5])}

In [103]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [105]:
bob._objects

{8229148789: tensor([1, 2, 3, 4, 5])}

In [106]:
alice._objects

{27505637640: (Wrapper)>[PointerTensor | alice:27505637640 -> bob:8229148789]}

In [107]:
x

(Wrapper)>[PointerTensor | me:44575661182 -> alice:27505637640]

In [108]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:44575661182 -> alice:27505637640]

In [109]:
bob._objects

{}

In [110]:
alice._objects

{27505637640: tensor([1, 2, 3, 4, 5])}

In [111]:
x.move(bob)

(Wrapper)>[PointerTensor | me:75863155652 -> bob:44575661182]

In [112]:
x

(Wrapper)>[PointerTensor | me:75863155652 -> bob:44575661182]

In [113]:
bob._objects

{44575661182: tensor([1, 2, 3, 4, 5])}

In [114]:
alice._objects

{}


**Exercise:**

To avoid exposing gradients among participants, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
import syft as sy
import torch as th
from torch import nn, optim

In [0]:
# create workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [117]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
# create local datasets at Bob and Alice

bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [0]:
# Send copies of the linear model to alice and bob
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [0]:
# create two opimizers for Alice and Bob
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data

In [0]:

# move the models to the third worker 
alices_model.move(secureWorker)
bobs_model.move(secureWorker)

# aggreegate the models (their average)
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

# --- use model.weight.data to access the weights, and model.bias.data to access bias
# --- use model.weight.set_(new_weights) to update the weights
# --- use model.bias.set_(new_bias) to update the bias

In [125]:
# make predictions uding the aggregated model
preds = model(data)

print(preds)

tensor([[0.2820],
        [0.2887],
        [0.5080],
        [0.5147]], grad_fn=<AddmmBackward>)
